In [2]:
# By Adam Gregg

import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Reading the data into a dataframe
df = pd.read_csv('o-ring-erosion-or-blowby.csv',
                   names = ['Number of O-rings', 'Number Experiencing Thermal Distress', 'Launch Temperature (F)', 
                            'Leak-check Pressure (psi)', 'Temporal Order of Flight'])
df.head()

,Number of O-rings,Number Experiencing Thermal Distress,Launch Temperature (F),Leak-check Pressure (psi),Temporal Order of Flight
0,6,0,66,50,1
1,6,1,70,50,2
2,6,0,69,50,3
3,6,0,68,50,4
4,6,0,67,50,5


In [4]:
# Dropping the columns that will not contribute to the regression model
df = df.drop(['Number of O-rings', 'Temporal Order of Flight'], axis = 1)
df.head()

,Number Experiencing Thermal Distress,Launch Temperature (F),Leak-check Pressure (psi)
0,0,66,50
1,1,70,50
2,0,69,50
3,0,68,50
4,0,67,50


In [5]:
# In order to predict the number of O-rings that will experience thermal distress for a given flight when the launch 
# temperature is below freezing, our regression model will consist of the independent variables being Launch Temperature 
# and Leak-check pressure while the Number Experiencing Thermal Distress will be our dependent variable, 
# i.e. our target variable.

# Let's first use the statsmodels OLS regression model
Y = df['Number Experiencing Thermal Distress']
X = df[['Launch Temperature (F)', 'Leak-check Pressure (psi)']]

# Adding a constant to include intercept
X = sm.add_constant(X)
reg_model = sm.OLS(Y, X).fit()
reg_model.params
reg_model.summary()

const                        3.329831
Launch Temperature (F)      -0.048671
Leak-check Pressure (psi)    0.002939
dtype: float64

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     Number Experiencing Thermal Distress   R-squared:                       0.354
Model:                                              OLS   Adj. R-squared:                  0.290
Method:                                   Least Squares   F-statistic:                     5.490
Date:                                  Tue, 31 Jul 2018   Prob (F-statistic):             0.0126
Time:                                          22:03:14   Log-Likelihood:                -17.408
No. Observations:                                    23   AIC:                             40.82
Df Residuals:                                        20   BIC:                             44.22
Df Model:                                             2                                         
Covariance Type:                              nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         3.3298      1.188      2.803      0.011       0.851       5.808
Launch Temperature (F)       -0.0487      0.017     -2.910      0.009      -0.084      -0.014
Leak-check Pressure (psi)     0.0029      0.002      1.699      0.105      -0.001       0.007
==============================================================================
Omnibus:                       19.324   Durbin-Watson:                   2.390
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.471
Skew:                           1.782   Prob(JB):                     8.00e-06
Kurtosis:                       6.433   Cond. No.                     1.84e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.84e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
# For the prediction the following formula will be used:
# Y = 31 * the temperature coefficient + pressure * the pressure coefficient + the constant
constant = reg_model.params[0]
temp_coef = reg_model.params[1]
pressure_coef = reg_model.params[2]
pressure_vals = [0, 50, 100, 200]

for i in pressure_vals:
    Y = 31*temp_coef + i*pressure_coef + constant
    print(('Number of O-rings in distress at {0} psi and 31 degrees F: {1:.2f}').format(i, Y))

Number of O-rings in distress at 0 psi and 31 degrees F: 1.82
Number of O-rings in distress at 50 psi and 31 degrees F: 1.97
Number of O-rings in distress at 100 psi and 31 degrees F: 2.11
Number of O-rings in distress at 200 psi and 31 degrees F: 2.41


In [7]:
# Let's now use the sklearn linear Regression model to compare results
Y = df['Number Experiencing Thermal Distress']
X = df[['Launch Temperature (F)', 'Leak-check Pressure (psi)']]
lm = LinearRegression()
lm.fit(X, Y)
lm.coef_
lm.intercept_

# Using the same formula as before for the prediction
constant = lm.intercept_
temp_coef = lm.coef_[0]
pressure_coef = lm.coef_[1]
pressure_vals = [0, 50, 100, 200]

for i in pressure_vals:
    Y = 31*temp_coef + i*pressure_coef + constant
    print(('Number of O-rings in distress at {0} psi and 31 degrees F: {1:.2f}').format(i, Y))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

array([-0.04867109,  0.00293932])

3.329830717693318

Number of O-rings in distress at 0 psi and 31 degrees F: 1.82
Number of O-rings in distress at 50 psi and 31 degrees F: 1.97
Number of O-rings in distress at 100 psi and 31 degrees F: 2.11
Number of O-rings in distress at 200 psi and 31 degrees F: 2.41


In conclusion, both models predict that approximately 2 O-rings will show erosion or blowby when the temperature is 31 degrees F even when the pressure is in the range of 0-200 psi. 